In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/Benign.zip" -d "/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/Benign"

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/Adware.zip" -d "/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/Adware"

In [ ]:
!unzip "/content/drive/MyDrive/Đồ án/SMS.zip" -d "/content/drive/MyDrive/Đồ án/SMS"

In [ ]:
!unzip "/content/drive/MyDrive/Đồ án/Riskware.zip" -d "/content/drive/MyDrive/Đồ án/Riskware"

In [ ]:
!unzip "/content/drive/MyDrive/Đồ án/Banking.zip" -d "/content/drive/MyDrive/Đồ án/Banking"

In [ ]:
import json

def read_json(filename):
  file_json = json.load(open(filename))
  static_analysis = file_json['Static_analysis']
  return (file_json['Pre_static_analysis']['Filename'], static_analysis['Permissions'], static_analysis['Opcodes'], static_analysis['API calls'], static_analysis['System commands'], static_analysis['Intents'])


In [ ]:
import os

def get_json_paths(path_to_json_dir):
  return [pos_json for pos_json in os.listdir(path_to_json_dir) if pos_json.endswith('.json')]


In [ ]:
permissions = set()
opcodes = set()
api_calls = set()
system_commands = set()
intents = set()

In [ ]:
root_dir = "/content/drive/MyDrive/Đồ án/"
dirs = ['Benign', 'SMS', 'Riskware', 'Banking', 'Adware']
for dir in dirs:
  dir_path = os.path.join(root_dir, dir)
  jsons = get_json_paths(dir_path)
  for j in jsons:
    (_, permission, opcode, api_call, system_command, intent) = read_json(os.path.join(dir_path, j))
    for p in permission:
      permissions.add(p.upper())
    for o in opcode: #for lấy key
      opcodes.add(o)
    for a in api_call: #for lấy key
      api_calls.add(a)
    for c in system_command: #for lấy key
      system_commands.add(c)
    for i in intent: #for lấy key
      intents.add(i)

In [ ]:
def write_file(name, data):
  f = open(name, "w")
  for item in data:
    f.write(item + "\n")
  f.close()

In [ ]:
write_file(os.path.join(root_dir, "permissions.txt"), permissions)
write_file(os.path.join(root_dir, "opcodes.txt"), opcodes)
write_file(os.path.join(root_dir, "api_calls.txt"), api_calls)
write_file(os.path.join(root_dir, "system_commands.txt"), system_commands)
write_file(os.path.join(root_dir, "intents.txt"), intents)

In [ ]:
def read_file(name):
  f = open(name, "r")
  output = set()
  myline = f.readline().strip()
  while myline:
    output.add(myline)
    myline = f.readline().strip()
  f.close()
  return output

In [ ]:
# tổng các đặc trưng (tên của api, của per, ...)
permissions = read_file('/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/permissions.txt')
opcodes = read_file('/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/opcodes.txt')
api_calls = read_file('/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/api_calls.txt')
system_commands = read_file('/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/system_commands.txt')
intents = read_file('/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/intents.txt')

In [ ]:
# chuyển đặc trưng của các file => dạng số
import pandas as pd
import numpy as np

root_dir = "/content/drive/MyDrive/Colab Notebooks/MalDroid 2020/static Features file JSON/"
# dirs = ['Benign', 'SMS', 'Riskware', 'Banking', 'Adware']
dirs = ['Benign', 'Adware']
header_permissions = ["file_name"] + list(permissions)
header_opcodes = ["file_name"] + list(opcodes)
header_api_calls = ["file_name"] + list(api_calls)
header_system_commands = ["file_name"] + list(system_commands)
header_intents = ["file_name"] + list(intents)

for dir in dirs:
  dir_path = os.path.join(root_dir, dir)
  jsons = get_json_paths(dir_path)

  df_permissions = pd.DataFrame([], columns=header_permissions)
  df_opcodes = pd.DataFrame([], columns=header_opcodes)
  df_api_calls = pd.DataFrame([], columns=header_api_calls)
  df_system_commands = pd.DataFrame([], columns=header_system_commands)
  df_intents = pd.DataFrame([], columns=header_intents)

  length = len(jsons)
  for idx, j in enumerate(jsons):
    print("%d/%d" % (idx, length))
    ex_df_permissions = pd.DataFrame([[0]*len(header_permissions)], columns=header_permissions) # khởi tạo 1 dòng có độ dài = tổng permission, có cột có header là tên các permission
    ex_df_opcodes = pd.DataFrame([[0]*len(header_opcodes)], columns=header_opcodes)
    ex_df_api_calls = pd.DataFrame([[0]*len(header_api_calls)], columns=header_api_calls)
    ex_df_system_commands = pd.DataFrame([[0]*len(header_system_commands)], columns=header_system_commands)
    ex_df_intents = pd.DataFrame([[0]*len(header_intents)], columns=header_intents)

    (file_name, permission, opcode, api_call, system_command, intent) = read_json(os.path.join(dir_path, j))
    ex_df_permissions["file_name"][0] = ex_df_opcodes["file_name"][0] = ex_df_api_calls["file_name"][0] = ex_df_system_commands["file_name"][0] = ex_df_intents["file_name"][0] = file_name
    for p in permission: #đánh trọng số
      if p != '':
        ex_df_permissions[p.upper()][0] = 1
    for o in opcode: #đánh trọng số
      ex_df_opcodes[o][0] = opcode[o]
    for a in api_call: # duyệt cấc api của từng file apk
      ex_df_api_calls[a][0] = api_call[a]
    for c in system_command: #đánh trọng số
      ex_df_system_commands[c][0] = system_command[c]
    for i in intent: #đánh trọng số
      if i != '':
        ex_df_intents[i][0] = intent[i]
    
    df_permissions = pd.concat([df_permissions, ex_df_permissions], ignore_index=True)
    df_opcodes = pd.concat([df_opcodes, ex_df_opcodes], ignore_index=True)
    df_api_calls = pd.concat([df_api_calls, ex_df_api_calls], ignore_index=True)
    df_system_commands = pd.concat([df_system_commands, ex_df_system_commands], ignore_index=True)
    df_intents = pd.concat([df_intents, ex_df_intents], ignore_index=True)

  df_permissions.to_csv(os.path.join(root_dir, dir + "_permissions.csv"), index=True)
  df_opcodes.to_csv(os.path.join(root_dir, dir + "_opcodes.csv"), index=True)
  df_api_calls.to_csv(os.path.join(root_dir, dir + "_api_calls.csv"), index=True)
  df_system_commands.to_csv(os.path.join(root_dir, dir + "_system_commands.csv"), index=True)
  df_intents.to_csv(os.path.join(root_dir, dir + "_intents.csv"), index=True)